In [2]:
import openai
import os
import datetime
from openai import OpenAI
import argparse
import pandas as pd
import json
import re
from tqdm import tqdm
import csv

def test_diagnose(each_diagnosis,true_diagnosis):
    return each_diagnosis



# parse the prediction
def segement_predict(predict):

    prediction = predict.replace('\n\n','\n')
    pattern = r'(\d+)\.\s*([^\d\n]+(?:\n(?!\d+\.)[^\d\n]+)*)'
    diagnosis_list = re.findall(pattern, prediction)
    processed_list = [diagnosis.strip() for number, diagnosis in diagnosis_list if int(number) <= 10]

    if len(processed_list) < 10:
        processed_list = prediction.split('\n')

    if len(processed_list) > 10:
            processed_list = processed_list[-10:]

    return processed_list



def main(file_name):
    save_path = file_name.split('.jsonl')[0]

    tltle = ['ID', 'case_report', 'gold', 'predict', 'topic', 'top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10']
    with open(f'{save_path}_test.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(tltle)

    data = []
    with open(file_name, 'r', encoding='utf-8') as file:
        for line in file:
            item = json.loads(line.strip())
            data.append(item)

    print('-'*100)
    acc_dic = {}
    error_list = []
    for i in tqdm(data, ncols=100):
        diagnosis = i['true']
        predict = i['predict']

        processed_list = segement_predict(predict)
        if len(processed_list) < 10:
            error_list.append(i)

        i['acc'] = []
        for prediction in processed_list:
            acc = test_diagnose(prediction, diagnosis)
            i['acc'].append(acc)

        if len(i['acc']) >10:
            i['acc'] = i['acc'][-10:]
        elif len(i['acc']) < 10:
            i['acc'] = i['acc'] + [0]*(10-len(i['acc']))


        entry = [i['id'], i['input'], i['true'], i['predict'], i['topic']] + i['acc']
        with open(f'{save_path}_1111111111111.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(entry)


    print(f'Number of daignoses less than 10: {len(error_list)}')
    with open(f'{save_path}_error.json', 'w', encoding='utf-8') as file:
        json.dump(error_list, file, ensure_ascii=False, indent=4)


In [3]:

name = 'Llama-3.3-70B-Instruct_JAMAFinalAll'
file_path = f'/home/gy237/project/llama3/total_final_test/Llama3.1_final_test/{name}.jsonl'

print('file_path:', file_path)
main(file_path)

file_path: /home/gy237/project/llama3/total_final_test/Llama3.1_final_test/Llama-3.3-70B-Instruct_JAMAFinalAll.jsonl
----------------------------------------------------------------------------------------------------


100%|███████████████████████████████████████████████████████████| 498/498 [00:00<00:00, 3069.06it/s]

Number of daignoses less than 10: 0
